In [77]:
import requests
import pandas as pd

# For normal

In [5]:
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport

# Select your transport with a defined url endpoint
transport = AIOHTTPTransport(url="https://countries.trevorblades.com/")

# Create a GraphQL client using the defined transport
client = Client(transport=transport, fetch_schema_from_transport=True)

# Provide a GraphQL query


# Execute the query on the transport
result = client.execute(query)
print(result)


AssertionError: Cannot run client.execute(query) if an asyncio loop is running. Use 'await client.execute_async(query)' instead.

# Async for ipython

In [40]:
import asyncio

from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport

transport = AIOHTTPTransport(url="https://api.thegraph.com/subgraphs/name/connext/nxtp-matic")

## Graph Queries

### Transactions from User

In [41]:
query = gql(
    """
query GetSenderTransactions($userId: String!, $sendingChainId: BigInt!, $status: TransactionStatus) {
  transactions(
    where: {user: $userId, status: $status, sendingChainId: $sendingChainId}
    orderBy: preparedBlockNumber
    orderDirection: desc
  ) {
    id
    status
    chainId
    preparedTimestamp
    user {
      id
    }
    router {
      id
    }
    receivingChainTxManagerAddress
    sendingAssetId
    receivingAssetId
    sendingChainFallback
    receivingAddress
    callTo
    sendingChainId
    receivingChainId
    callDataHash
    transactionId
    amount
    expiry
    preparedBlockNumber
    bidSignature
    prepareCaller
    fulfillCaller
    cancelCaller
    prepareTransactionHash
    fulfillTransactionHash
    cancelTransactionHash
  }
}
"""
)

params = {
        "sendingChainId": 137,
        "userId": "0x4752a32843b661fedc525c979aaa01fa54d31366",
        "status": "Fulfilled"
    }

In [101]:
query = gql(
    """
query fetchAllTransactions($batch: Int!) {
  transactions (
    first:100, skip: $batch,
    orderBy: preparedBlockNumber
    orderDirection: asc
  ){
    id
    status
    chainId
    preparedTimestamp
    user {
      id
    }
    router {
      id
    }
    receivingChainTxManagerAddress
    sendingAssetId
    receivingAssetId
    sendingChainFallback
    receivingAddress
    callTo
    sendingChainId
    receivingChainId
    callDataHash
    transactionId
    amount
    expiry
    preparedBlockNumber
    bidSignature
    prepareCaller
    fulfillCaller
    cancelCaller
    prepareTransactionHash
    fulfillTransactionHash
    cancelTransactionHash
  }
}
"""
)

params = {
        "batch": 0
    }

## Querying

In [102]:
transactions = pd.DataFrame(columns=['amount', 'bidSignature', 'callDataHash', 'callTo', 'cancelCaller', 'cancelTransactionHash', 'chainId', 'expiry', 'fulfillCaller', 'fulfillTransactionHash', 'id', 'prepareCaller', 'prepareTransactionHash', 'preparedBlockNumber', 'preparedTimestamp', 'receivingAddress', 'receivingAssetId', 'receivingChainId', 'receivingChainTxManagerAddress', 'router', 'sendingAssetId', 'sendingChainFallback', 'sendingChainId', 'status', 'transactionId', 'user'])

In [105]:
async def graph_query(query, params):

    # Using `async with` on the client will start a connection on the transport
    # and provide a `session` variable to execute queries on this connection
    async with Client(
        transport=transport, fetch_schema_from_transport=True,
    ) as session:

        # Execute single query
        for batch in range(47):
            result = await session.execute(query, variable_values=params)
            for tr in result["transactions"]:
                list_values = list(tr.values())
                list_values[19] = list_values[19]["id"]
                list_values[25] = list_values[25]["id"]
                transactions.loc[len(transactions.index)] = list_values
            params['batch']+=100
            print(params["batch"])
            break
        return result

result = await graph_query(query, params)

4700


In [106]:
transactions.shape

(4625, 26)

In [110]:
transactions.user.nunique()

2773

In [110]:
transactions.user.nunique()

2773

In [67]:
len(result["transactions"])

100

In [ ]:
transactions.loc[len(transactions.index)] = 

In [80]:
li = list(result["transactions"][0].values())

In [90]:
li

['998000000',
 '0x7420a1e101bed6fe118f260ace39d5432725fde60b37e8ccb2ad9eeda82e28e24d56aa658d3a1aeae106c89e883f87a74862c640bcc52ab1af4109609d4e86611c',
 '0xc5d2460186f7233c927e7db2dcc703c0e500b653ca82273b7bfad8045d85a470',
 '0x0000000000000000000000000000000000000000',
 None,
 None,
 '137',
 '1630862168',
 '0xe1dd7b28ab085978ead65aff92b22b6322a96422',
 '0x886b855af0c48a953a45a87ebb0be3cc9c6a7c921fcf809a4a65ca23a2184953',
 '0x971c3f60297a991c39bf20c3a0bef15cb3dd8eecaf41b4a59ceb0cde4621a14a-0xd8faec528b9fbf4fe3226d6beb8b14a4f9bad655-0xe1dd7b28ab085978ead65aff92b22b6322a96422',
 '0xd8faec528b9fbf4fe3226d6beb8b14a4f9bad655',
 '0x7a5710efe6fecfb1c1c261187ca30181ed05f3d5e9c96d3cf4edaeaa708dd867',
 '18670139',
 '1630603047',
 '0xd8faec528b9fbf4fe3226d6beb8b14a4f9bad655',
 '0x049d68029688eabf473097a2fc38ef61633a3c7a',
 '250',
 '0x31efc4aeaa7c39e54a33fdc3c46ee2bd70ae0a09',
 '0xe1dd7b28ab085978ead65aff92b22b6322a96422',
 '0xc2132d05d31c914a87c6611c10748aeb04b58e8f',
 '0xd8faec528b9fbf4fe3226d6beb

In [88]:
li[25]

{'id': '0xd8faec528b9fbf4fe3226d6beb8b14a4f9bad655'}

In [85]:
li[19] = li[19]["id"]

In [89]:
li[25] = li[25]["id"]

In [75]:
result["transactions"][0].keys()

dict_keys(['amount', 'bidSignature', 'callDataHash', 'callTo', 'cancelCaller', 'cancelTransactionHash', 'chainId', 'expiry', 'fulfillCaller', 'fulfillTransactionHash', 'id', 'prepareCaller', 'prepareTransactionHash', 'preparedBlockNumber', 'preparedTimestamp', 'receivingAddress', 'receivingAssetId', 'receivingChainId', 'receivingChainTxManagerAddress', 'router', 'sendingAssetId', 'sendingChainFallback', 'sendingChainId', 'status', 'transactionId', 'user'])

In [68]:
result

{'transactions': [{'amount': '998000000',
   'bidSignature': '0x7420a1e101bed6fe118f260ace39d5432725fde60b37e8ccb2ad9eeda82e28e24d56aa658d3a1aeae106c89e883f87a74862c640bcc52ab1af4109609d4e86611c',
   'callDataHash': '0xc5d2460186f7233c927e7db2dcc703c0e500b653ca82273b7bfad8045d85a470',
   'callTo': '0x0000000000000000000000000000000000000000',
   'cancelCaller': None,
   'cancelTransactionHash': None,
   'chainId': '137',
   'expiry': '1630862168',
   'fulfillCaller': '0xe1dd7b28ab085978ead65aff92b22b6322a96422',
   'fulfillTransactionHash': '0x886b855af0c48a953a45a87ebb0be3cc9c6a7c921fcf809a4a65ca23a2184953',
   'id': '0x971c3f60297a991c39bf20c3a0bef15cb3dd8eecaf41b4a59ceb0cde4621a14a-0xd8faec528b9fbf4fe3226d6beb8b14a4f9bad655-0xe1dd7b28ab085978ead65aff92b22b6322a96422',
   'prepareCaller': '0xd8faec528b9fbf4fe3226d6beb8b14a4f9bad655',
   'prepareTransactionHash': '0x7a5710efe6fecfb1c1c261187ca30181ed05f3d5e9c96d3cf4edaeaa708dd867',
   'preparedBlockNumber': '18670139',
   'preparedTim